In [1]:
# Import libraries
import cudf
import pandas as pd
from numba import cuda
import shapefile as sh
import numpy as np
import math

In [2]:
# Read in US Tiger Line Edge shapefile (Example here is King County)
shp_name = "/data/us_tiger/king_county/tl_2018_53033_edges.shp"
sf = sh.Reader(shp_name)

In [3]:
# Get field positions for road type and nodes
fields = list(zip(*sf.fields))
mtfcc_pos = fields[0].index('MTFCC') - 1
roadflg_pos = fields[0].index('ROADFLG') - 1
nodef_pos = fields[0].index('TNIDF') - 1
nodet_pos = fields[0].index('TNIDT') - 1

In [ ]:
# Parse shape file into list of edges and their constituent vectors
vectors = []
edges = []
# Read nodes into a dictionary to de-duplicate
nodes = {}
node_num = 0
mtfcc = []
# Iterate through shapefile shape objects
for i,geo in enumerate(sf.shapes()):
    # Get the shape attributes
    attr = sf.record(i)
    # Check that the edge is a road
    if attr[roadflg_pos] == 'Y':
        # Test if mtfcc (edge classification) is in list, if not add it
        if attr[mtfcc_pos] not in mtfcc:
            mtfcc += [attr[mtfcc_pos]]
        # Test if from node exists
        if attr[nodef_pos] not in nodes:
            # If not, add it to the dictionary with coordinates of first point of vector
            nodes[attr[nodef_pos]] = [node_num,geo.points[0][0],geo.points[0][1]]
            node_num += 1
        # Test if to node exists
        if attr[nodet_pos] not in nodes:
            # If not, add it to the dictionary with coordinates of last point of vector
            nodes[attr[nodet_pos]] = [node_num,geo.points[-1][0],geo.points[-1][1]]
            node_num += 1
        # Add edge to table 
        edges += [[i,nodes[attr[nodef_pos]][0],nodes[attr[nodet_pos]][0],attr[mtfcc_pos]]]
        # Add edge to table with from and too nodes reversed
        edges += [[i,nodes[attr[nodet_pos]][0],nodes[attr[nodef_pos]][0],attr[mtfcc_pos]]]
        # Unravel and serialise the vector points into a list
        for p in geo.points:
            vectors += [[i,p[0],p[1]]]
# Sort mtfcc
mtfcc.sort()

In [ ]:
# Zip the vector points and copy to cudf DataFrame
vz = list(zip(*vectors))
vector_df = cudf.DataFrame()
vector_df['id'] = vz[0]
vector_df['lon'] = vz[1]
vector_df['lat'] = vz[2]

In [ ]:
# Zip the node points and copy to cudf DataFrame
nz = list(zip(*sorted(nodes.values())))
node_df = cudf.DataFrame()
node_df['id'] = nz[0]
node_df['lon'] = nz[1]
node_df['lat'] = nz[2]

In [ ]:
# Zip the edges and copy to cudf DataFrame
ez = list(zip(*edges))
edge_df = cudf.DataFrame()
edge_df['id'] = ez[0]
edge_df['src'] = ez[1]
edge_df['dst'] = ez[2]
edge_df['mtfcc'] = [mtfcc.index(m) for m in ez[3]]

In [ ]:
# Check results on vector Dataframe
print(vector_df)

In [ ]:
# Check results on node Dataframe
print(node_df)

In [ ]:
# Check results on edge DataFrame
print(edge_df)

In [ ]:
# Define chunk size and parameters
threads_per_block = 128
trunk_size = 10240
data_length = vector_df['id'].count()

In [ ]:
# Haversine function for lengths
@cuda.jit(device=True)
def haversine_distance(lon_1, lat_1, lon_2, lat_2):
    lon_1 = lon_1 * math.pi / 180.0 
    lon_2 = lon_2 * math.pi / 180.0 
    lat_1 = lat_1 * math.pi / 180.0 
    lat_2 = lat_2 * math.pi / 180.0
    d_lon = lon_2 - lon_1 
    d_lat = lat_2 - lat_1 
    a = math.sin(d_lat/2)**2 + math.cos(lat_1) * math.cos(lat_2) * math.sin(d_lon/2)**2
    return 2 * math.asin(math.sqrt(a)) * 6371000.0

In [ ]:
# Window function to calculate distance between adjacent points in feet
def adjacent_distance(id, lon, lat, dist):
    for i in range(cuda.threadIdx.x, id.size, cuda.blockDim.x):
        # If the first point of vector return zero length
        if i == 0:
            dist[i] = 0.0
        # Or if the vector changes (1st point of new vector)
        elif id[i] != id[i-1]:
            dist[i] = 0.0
        else:
            dist[i] = haversine_distance(lon[i],lat[i],lon[i-1],lat[i-1]) * 3.280839895013123

In [ ]:
%%time
# Apply the window function
vector_df = vector_df.apply_chunks(adjacent_distance,
                     incols=['id','lon','lat'],
                     outcols=dict(dist=np.float64),
                     kwargs=dict(),
                     chunks=list(range(0, data_length,
                                       trunk_size))+ [data_length],
                     tpb=threads_per_block)

In [ ]:
# Check the result
print(vector_df)

In [ ]:
%%time
# Group by and sum the distances for each edge
distance_df = vector_df[['id','dist']].groupby(['id']).sum()
distance_df['id'] = distance_df.index

In [ ]:
%%time
# Merge the distances into the edge DataFrame to produce graph
edge_df = edge_df.merge(distance_df,on=['id'], how='left').sort_values('id')

In [ ]:
# Check Result
print(edge_df)

In [ ]:
# Save road graph to csv
graph_file = shp_name[:-4] + "_graph.csv"
edge_df.to_pandas().to_csv(graph_file, index=False)

In [ ]:
# Save road modes to csv
node_file = shp_name[:-4] + "_nodes.csv"
node_df.to_pandas().to_csv(node_file, index=False)

In [ ]:
# print mtfcc table (used to filter on road type)
print([(i,m) for i,m in enumerate(mtfcc)])